# Plots in section 3: "Results and discussion"

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
import itertools

rc = {"font.family" : "serif", 
      "mathtext.fontset" : "stix"}
plt.rcParams.update(rc)
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]

if not os.path.isdir('Figures'):
        os.mkdir('Figures')

def assign_styles(unique_values):
    """
    Assign each value in 'unique_values' a marker, color, filling and line style.
    Returns mappings as four dictionaries.
    """
    unique_values.sort()

    markers = itertools.cycle(['o', 'o', 's', 's'])
    colors = itertools.cycle(['steelblue', 'steelblue', 'orange', 'orange', 'hotpink', 'sandybrown'] + \
                 [color for color in mcolors.CSS4_COLORS])
    filling = itertools.cycle(['full', 'none', 'full', 'none'])
    lines = itertools.cycle(['dotted', 'dashed', 'dotted', 'dashed'])

    markerdict = dict()
    linestyledict = dict()
    colordict = dict()
    fillingdict = dict()

    for value in unique_values:
        markerdict[value] = next(markers)
        linestyledict[value] = next(lines)
        colordict[value] = next(colors)
        fillingdict[value] = next(filling)

    return markerdict, linestyledict, colordict, fillingdict

def assign_styles_Q0(unique_values):
    """
    Assign each value in 'unique_values' a marker, color, filling and line style.
    Returns mappings as four dictionaries.
    """
    unique_values.sort()

    #markers = itertools.cycle(['o', 'o', 's', 's'])
    markers = itertools.cycle(['o', 's', '^', '+', 'x', '*'])
    #colors = itertools.cycle(['steelblue', 'steelblue', 'orange', 'orange', 'hotpink', 'sandybrown'] + \
    #             [color for color in mcolors.CSS4_COLORS])
    colors = itertools.cycle(['steelblue', 'orange', 'mediumseagreen', 'palevioletred', 'grey', 'crimson'] + \
                 [color for color in mcolors.CSS4_COLORS])
    filling = itertools.cycle(['full', 'none', 'full', 'none'])
    lines = itertools.cycle(['dotted', 'dashed', 'dashdot', (0, (3, 1, 1, 1))])

    markerdict = dict()
    linestyledict = dict()
    colordict = dict()
    fillingdict = dict()

    for value in unique_values:
        markerdict[value] = next(markers)
        linestyledict[value] = next(lines)
        colordict[value] = next(colors)
        fillingdict[value] = next(filling)

    return markerdict, linestyledict, colordict, fillingdict

## Plots for flow rate distribution

In [ ]:
# Prepare data
path = 'Data/Data_interface_velocity.csv'
df = pd.read_csv(path, sep=';')

index_columns = ['geometry']
df.set_index(index_columns, inplace=True)
df.sort_index(inplace=True)

# For crevice width w=0.4mm
df = df.loc[['w0.4_r0.1', 'w0.4_r0.2']]

# For crevice width w=0.3mm
#df = df.loc[['w0.3_r0.1', 'w0.3_r0.2']]

group_by_parameters = list(index_columns)
unique_values = list(df.index.unique(level='geometry'))
markers, linestyles, colors, filling = assign_styles(unique_values)

from matplotlib.pyplot import figure
figure(figsize=(3.5, 3.5), dpi=300)

# Creat plot
for params, subsetdf in  df.groupby(level=group_by_parameters):
    
    labels = params.split('_')[1]
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['q_crevice'].values/subsetdf['q_upstream'].values, yerr=subsetdf['u_crevice_std'].values, fmt='o', fillstyle=filling[params], color='steelblue', capsize=3, label=str(labels)+'-crevice')
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['q_downstream'].values/subsetdf['q_upstream'].values, yerr=subsetdf['u_downstream_std'].values, fmt='o', fillstyle=filling[params], color='orange', capsize=3, label=str(labels)+'-downstream')
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['q_upstream'].values/subsetdf['q_upstream'].values, yerr=subsetdf['u_upstream_std'].values, fmt='o', fillstyle=filling[params], color='darkcyan', capsize=3, label=str(labels)+'-upstream')
    plt.gca().xaxis.set_minor_locator(ticker.FixedLocator(subsetdf['flow_rate']))
    plt.xlabel('Volumetric flow rate $Q_0$ [ml/s]')
    plt.ylabel('$Q/Q_0$ [-]')
    plt.ylim([-0.2, 1.5])
    plt.legend(loc='center right', bbox_to_anchor=(1.7, 0.5), fontsize=12)

plt.grid(which='both')
plt.savefig("Figures/Flow_rate_distribution_w0.4.jpg", dpi=1200, bbox_inches="tight")

## Plot for flow rate error $\delta$

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(3.5, 3.5), dpi=1200)

# Prepare data
path = 'Data/Data_interface_velocity.csv'
df = pd.read_csv(path, sep=';')

index_columns = ['geometry']
df.set_index(index_columns, inplace=True)
df.sort_index(inplace=True)

group_by_parameters = list(index_columns)
unique_values = list(df.index.unique(level='geometry'))
markers, linestyles, colors, filling = assign_styles(unique_values)

# Create plot
for params, subsetdf in  df.groupby(level=group_by_parameters):
    
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['q_error'].values, yerr=0 , fmt=markers[params], fillstyle=filling[params], color=colors[params], capsize=3, label=params)
    plt.xlabel('Volumetric flow rate $Q_0$ [ml/s]')
    plt.ylabel('Flow rate error $\delta$ [-]')
    plt.gca().xaxis.set_minor_locator(ticker.FixedLocator(subsetdf['flow_rate']))
    plt.legend()

plt.grid(which='both')
plt.savefig("Figures/Flow_rate_error.jpg", dpi=1200, bbox_inches="tight")

## Plot for penetration depth $D$
The dotted and dashed lines are the linear fits between $D$ and $1/Q_0$.

In [ ]:
# Prepare data
path = 'Data/Data_others.csv'
df = pd.read_csv(path, sep=';')

index_column = ['geometry']
df.set_index(index_column, inplace=True)
df.sort_index(inplace=True)

group_by_parameters = list(index_column)
unique_values = list(df.index.unique(level='geometry'))
markers, linestyles, colors, filling = assign_styles(unique_values)

from matplotlib.pyplot import figure
figure(figsize=(3.5, 3.5), dpi=1200)

# Create plot
for params, subsetdf in  df.groupby(level=group_by_parameters):
    
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['depth_mean'].values*1000, yerr=subsetdf['depth_std'].values, fmt=markers[params], fillstyle=filling[params], color=colors[params], capsize=3, label=params)
    # Apply linear fits regression between dx and 1/Q0
    linear_model = np.polyfit(1/subsetdf['flow_rate'].values, subsetdf['depth_mean'].values*1000, 1)
    linear_model_fn = np.poly1d(linear_model)
    x_s = np.linspace(0.00015, 0.006, 100)
    plt.plot(x_s, linear_model_fn(1/x_s), color=colors[params], linestyle=linestyles[params], label=' ')
    plt.gca().xaxis.set_minor_locator(ticker.FixedLocator(subsetdf['flow_rate']))
    plt.xlabel('Volumetric flow rate $Q_0$ [ml/s]')
    plt.ylabel('Penetration depth $D$ [mm]')
    plt.legend(ncol=2, columnspacing= -1)

plt.grid(which='both')
plt.savefig("Figures/Penetration_depth_D.jpg", dpi=1200, bbox_inches="tight")

## Plot for maximum pinning distance $dx$
The dotted and dashed lines are the linear fits between $dx$ and $Q_0$.

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(3.5, 3.5), dpi=1200)

# Create plot
for params, subsetdf in  df.groupby(level=group_by_parameters):
    
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['dx_mean'].values*1000, yerr=subsetdf['dx_std'].values, fmt=markers[params], fillstyle=filling[params], color=colors[params], capsize=3, label=params)
    # Apply linear fits regression between dx and Q0
    linear_model = np.polyfit(subsetdf['flow_rate'].values, subsetdf['dx_mean'].values*1000, 1)
    linear_model_fn = np.poly1d(linear_model)
    x_s = np.linspace(-0.0001, 0.0065, 100)
    plt.plot(x_s, linear_model_fn(x_s), color=colors[params], linestyle=linestyles[params], label=' ')
    plt.gca().xaxis.set_minor_locator(ticker.FixedLocator(subsetdf['flow_rate']))
    plt.xlabel('Volumetric flow rate $Q_0$ [ml/s]')
    plt.ylabel('Maximum pinning distance $dx$ [mm]')
    plt.legend(ncol=2, columnspacing= -1)

plt.grid(which='both')
plt.savefig("Figures/dx.jpg", dpi=1200, bbox_inches="tight")

## Plot for interface pinning time $dt$
The dotted and dashed lines are the linear fits between $dt$ and $1/Q_0$.

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(3.5, 3.5), dpi=1200)

# Create plot
for params, subsetdf in  df.groupby(level=group_by_parameters):
    
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['dt_mean'].values, yerr=subsetdf['dt_std'].values, fmt=markers[params], fillstyle=filling[params], color=colors[params], capsize=3, label=params)
    # Apply linear fits regression between dx and 1/Q0
    linear_model = np.polyfit(1/subsetdf['flow_rate'].values, subsetdf['dt_mean'].values, 1)
    linear_model_fn = np.poly1d(linear_model)
    x_s = np.linspace(0.0001, 0.006, 100)
    plt.plot(x_s, linear_model_fn(1/x_s), color=colors[params], linestyle=linestyles[params], label=' ')
    plt.gca().xaxis.set_minor_locator(ticker.FixedLocator(subsetdf['flow_rate']))
    plt.xlabel('Volumetric flow rate $Q_0$ [ml/s]')
    plt.ylabel('Pinning time $dt$ [s]')
    plt.yscale('log')
    plt.legend(ncol=2, columnspacing= -1)

plt.grid(which='both')
plt.savefig("Figures/Pinning_dt.jpg", dpi=1200, bbox_inches="tight")

## Plot for normalized interface pinning time $dt/T$
The dotted and dashed lines are the linear fits between $dt/T$ and $Q_0$.

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(3.5, 3.5), dpi=1200)

# Create plot
for params, subsetdf in  df.groupby(level=group_by_parameters):
    
    plt.errorbar(subsetdf['flow_rate'].values, subsetdf['dt_t_mean'].values, yerr=subsetdf['dt_t_std'].values, fmt=markers[params], fillstyle=filling[params], color=colors[params], capsize=3, label=params)
    # Apply linear fits regression between dx and Q0
    linear_model = np.polyfit(subsetdf['flow_rate'].values, subsetdf['dt_t_mean'].values, 0)
    linear_model_fn = np.poly1d(linear_model)
    x_s = np.linspace(0.0001, 0.006, 100)
    plt.plot(x_s, linear_model_fn(1/x_s), color=colors[params], linestyle=linestyles[params], label=' ')
    plt.gca().xaxis.set_minor_locator(ticker.FixedLocator(subsetdf['flow_rate']))
    plt.xlabel('Volumetric flow rate $Q_0$ [ml/s]')
    plt.ylabel('Normalized pinning time $dt/T$ [-]')
    plt.legend(ncol=2, columnspacing= -1)

plt.grid(which='both')
plt.savefig("Figures/Pinning_dt_t.jpg", dpi=1200, bbox_inches="tight")

## Plot for non-dimensional parameter $w/r$
The dashed lines are the polynomial fits between $w/r$ and different parameters.

In [ ]:
from matplotlib.pyplot import figure

path = 'Data/Data_others.csv'
df = pd.read_csv(path, sep=';')

index_column = ['flow_rate']
df.set_index(index_column, inplace=True)
df.sort_index(inplace=True)

group_by_parameters = list(index_column)
unique_values = list(df.index.unique(level='flow_rate'))
markers, linestyles, colors, filling = assign_styles_Q0(unique_values)

poly_degree = [3, 1, 3, 3]
data_mean = ['depth_mean', 'dx_mean', 'dt_mean', 'dt_t_mean']
data_std = ['depth_std', 'dx_std', 'dt_std', 'dt_t_std']
titles = ['D_wr', 'dx_wr', 'dt_wr', 'dt_t_wr']
labels = ['Penetration depth $D$ [m]', 'Maximum pinning distance $dx$ [m]', 
          'Pinning time $dt$ [s]', 'Normalized pinning time $dt/T$ [-]']

for i in range(len(poly_degree)):
    fig, ax = plt.subplots(figsize=(3.5, 3.5), dpi=1200)
    
    for params, subsetdf in  df.groupby(level=group_by_parameters):
        if i == 0 or i == 1:
            ax.errorbar(subsetdf['w_r'].values, subsetdf[data_mean[i]].values*1000, yerr=subsetdf[data_std[i]].values, fmt=markers[params], color=colors[params], capsize=3, label=str(params)+' ml/s')
            linear_model = np.polyfit(subsetdf['w_r'].values, subsetdf[data_mean[i]].values*1000, poly_degree[i])
        else:
            ax.errorbar(subsetdf['w_r'].values, subsetdf[data_mean[i]].values, yerr=subsetdf[data_std[i]].values, fmt=markers[params], color=colors[params], capsize=3, label=str(params)+' ml/s')
            linear_model = np.polyfit(subsetdf['w_r'].values, subsetdf[data_mean[i]].values, poly_degree[i])           
        linear_model_fn = np.poly1d(linear_model)
        x_s = np.linspace(1.4, 4, 100)
        ax.plot(x_s, linear_model_fn(x_s), color=colors[params], linestyle='--', label=' ')
        #plt.xticks(ticks=subsetdf['flow_rate'], minor=True)
        ax.set_xticks(ticks=subsetdf['w_r'])
        ax.set_xlabel(' $w/r$ [-]')
        ax.set_ylabel(labels[i])
        ax.grid(b=True, which='both')
        
        fig.savefig(os.path.join('Figures/', titles[i]+'.jpg'), bbox_inches="tight")